In [ ]:
import os, sys, re
import pandas as pd
import numpy as np

def probeix(df, vmin, vmax):
    return np.logical_and((df >= vmin).all(axis = 1), (df <= vmax).all(axis = 1))

def test(x, th = 0.5, vmin = 0, vmax = 1e6):
    s = probeix(x, vmin = vmin, vmax = vmax)
    test = s.sum() / len(s) >= th
#     ntrue  = s.sum()
#     nfalse = (s == False).sum()
    return test

def get_wrong_measurements(df, probe = None, desc = None, th = 0.5, vmin = 0, vmax = 1e6):
#     desc2 = desc.sort_index()
    ix = vels.groupby(level = desc.index.names).apply(test, vmin = vmin, vmax = vmax)
    if not np.logical_or(probe is None, desc is None):
        out = desc.loc[ix == False].query("probe == '%s'" %(probe))
        out = (out, ix)
    else:
        out = ix
    return out

def pprint(msg, msg_title = '', msg_decorator = '#', len_decorator = 40):
    nhead = len_decorator - len(msg_title) - 2
    if nhead <= 0:
        nhead = 1
        nfoot = len(msg_title) + 4
    else:
        nfoot = len_decorator
    
    top_decorator = msg_decorator * (nhead // 2) 
    print(top_decorator + ' ' + msg_title  +  ' ' + top_decorator, 
          msg, nfoot * '#' + '\n',
          sep = '\n')
    return
mix = pd.IndexSlice

In [ ]:
datapath = '/home/urlab/sandbox/data/characterization/autoscan/autoscan.h5'
savepath = '/home/urlab/Documents/'

# load the data
df = pd.read_hdf(datapath, key = 'data')
desc = pd.read_hdf(datapath, key = 'description')

In [ ]:
velcols = df.columns[-4:-2]
permcol = 'perm'
hammcol = 'e_star'

describe_cols = ['mean', 'std', 'min', 'max',]

In [ ]:
# set min and max expected for measurement
velmin = 0.5e3
velmax = 7.0e3

# check velocities
vels = df.loc[:, velcols]

## identify offending values and those to keep
velidx = probeix(vels, vmin = velmin, vmax = velmax)

## firtst check how they are distributed
pprint(vels.describe().apply(np.round, decimals = 2), 'summary of raw')

## check the data makes sense
pprint(vels.loc[velidx, :].describe().apply(np.round, decimals = 2), 'summary of correct measurements')

# get the labels that have problem
out, ix = get_wrong_measurements(vels, probe = 'vel', desc = desc, vmin = velmin, vmax = velmax)

# the ix returned from get_wrong_measurements considers only those measurements for which more than half the values were correct.
# samples that don't meet this criteria are lost. THis is different than probeix, which only considers the values, independent of the samples. 
## check the data makes sense
pprint(vels.loc[mix[ix, :]].describe().apply(np.round, decimals = 2), 'summary with correct samples')

In [ ]:
# vels.loc[mix[ix, :]]
ss = ix[ix]
s = ss.index.values
print(s.shape, ss.shape, velidx.shape, velidx.index.droplevel(6).unique().shape, ix.shape)

In [ ]:
tt = [t for t in s]
vv = [v for v in vels.index.droplevel(6).values]

d = np.zeros(len(vv))

for j in range(len(tt)):
    kappa = len(vv)
    for k in range(kappa):
        if vv[k] == tt[j]:
            d[k] = 1
d.shape


# vels.loc[mix[tt[0], :]]
# mix[s[0], :]

In [ ]:
x = [vels.index.values[k] for j in range(len(tt)) for k in range(len(vv)) if vv[k] == tt[j]]
len(x)
# vels.loc(axis = 0)[mix[ss.index[4]]]

In [ ]:
v = velidx.index.droplevel(6).values
s = ss.index.values
x = [vels.index[k] for j in range(len(s)) for k in range(len(v)) if v[k] == s[j]]

In [ ]:
indexer = vels.index[d == 1]
vels.loc[indexer].describe()

In [ ]:
perm = df.loc[:, [permcol]]
print(perm.describe())
permidx = probeix(perm, vmin = 0.3, vmax = 5e3)
## check the data makes sense
print(perm.loc[permidx, :].describe())

## print the labels that have problem
out, _  = get_wrong_measurements(perm, permidx, 'perm', desc)
out.sample(10)

In [ ]:
# remove offending values and keep just good measurements
idx = np.logical_and(velidx.values, permidx.values)
dc = df.loc[idx, :]

In [ ]:
# alternative, fill the values with means or nans
vels.loc[velidx == False, :] = np.nan
desc.loc[vels.index[velidx == False].droplevel(6).drop_duplicates()].query("probe == 'vel'")

In [ ]:
# x = velidx[velidx == False].index.droplevel(6).drop_duplicates()
# for t in x:
#     vels.loc[mix[[*t], :], :]

In [ ]:
# means = vels.groupby(level = velidx.index.names[:-1], sort = False).apply(np.mean)

# for dd in means.index:
#     pass
# dd

In [ ]:
# mix = pd.IndexSlice
# mix[[*ix], :]
# vels.loc(axis = 0)[mix[[dd], :], :]

In [ ]:
# def myquery(x, vels, velmin = 0.5e3, velmax = v):
#     v = vels.copy()
#     s = v.loc[x, :].shape[0]
#     idx = np.logical_and((v >= velmin).all(axis = 1), (v <= velmax).all(axis = 1))
#     skeep = np.sum(idx)
#     sdrop = np.sum(idx == False)
#     return s, skeep, sdrop

# for a, b in idx.loc[revise_idx, :].groupby(level = idx.index.names[2:-1]):
#     pass#.describe()

# x = revise_idx.droplevel(6).drop_duplicates()
# idx.loc[slice(x, :), :]

# idx.loc[revise_idx, :]
# idx.groupby(level = idx.index.names[2:-1]).describe()
# .loc[:, 'r'] = 'review'

# mix = pd.IndexSlice
# pd.concat((idx.loc[mix[[*t], :]] for t in x)) 

# mix = pd.IndexSlice
# pd.concat((idx.loc[mix[[*t], :]] for t in x)) 